In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt

# Load the MovieLens dataset
!wget -nc https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -n ml-latest-small.zip

ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

# Merge datasets
data = pd.merge(ratings, movies, on='movieId')
print(data.head())

# Create User-Item Matrix
user_item_matrix = data.pivot_table(index='userId', columns='title', values='rating')

# Fill NaN with 0 for similarity calculations
user_item_matrix_filled = user_item_matrix.fillna(0)

# Compute User Similarity (Cosine)
user_similarity = cosine_similarity(user_item_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Compute Item Similarity (Cosine)
item_similarity = cosine_similarity(user_item_matrix_filled.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

def get_user_based_recommendations(user_id, num_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:6]
    user_ratings = user_item_matrix.loc[similar_users].mean()
    user_ratings = user_ratings[user_ratings > 0]
    recommended_movies = user_ratings.sort_values(ascending=False).head(num_recommendations)
    return recommended_movies

def get_item_based_recommendations(movie_title, num_recommendations=5):
    similar_items = item_similarity_df[movie_title].sort_values(ascending=False)[1:num_recommendations+1]
    return similar_items

# Hybrid Recommender System
def hybrid_recommendation(user_id, movie_title, num_recommendations=5):
    user_recs = get_user_based_recommendations(user_id, num_recommendations)
    item_recs = get_item_based_recommendations(movie_title, num_recommendations)
    hybrid_recs = pd.concat([user_recs, item_recs]).sort_values(ascending=False).head(num_recommendations)
    return hybrid_recs

# Evaluation Metrics
def evaluate_recommendations(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')
    return precision, recall, f1

# Example Recommendations
print("User-based recommendations:")
print(get_user_based_recommendations(user_id=1))

print("Item-based recommendations:")
print(get_item_based_recommendations(movie_title='Toy Story (1995)'))

print("Hybrid recommendations:")
print(hybrid_recommendation(user_id=1, movie_title='Toy Story (1995)'))

# Example Evaluation
y_true = [1, 1, 0, 1, 0]  # Ground truth (actual liked movies)
y_pred = [1, 0, 1, 1, 1]  # Predicted recommendations
precision, recall, f1 = evaluate_recommendations(y_true, y_pred)
print(f"Precision: {precision}, Recall: {recall}, F1-score: {f1}")


--2025-03-25 01:06:53--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  6.13MB/s    in 0.2s    

2025-03-25 01:06:54 (6.13 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
   userId  movieId  rating  timestamp                        title  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men